In [ ]:
from google.colab import drive
import pandas as pd

In [ ]:
drive_dir = '/content/drive'
drive.mount(drive_dir, force_remount=True)

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/My Drive/CSCI2470_final_project/Suicide_Detection.csv"
df = pd.read_csv(data_path, )
print(df.head())

   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  I’m so lostHello, my name is Adam (16) and I’v...      suicide


In [ ]:
df['text'].apply(lambda x: len(x.split())).describe()

count    232074.000000
mean        131.925235
std         217.476787
min           1.000000
25%          26.000000
50%          60.000000
75%         155.000000
max        9684.000000
Name: text, dtype: float64

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# subset['tokenized'] = subset['text'].apply(lambda x: tokenizer.tokenize(x))
df['tokenized'] = df['text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
df['tokenized'].apply(lambda x: len(x)).describe()

count    232074.000000
mean        167.729237
std         293.175278
min           0.000000
25%          35.000000
50%          79.000000
75%         195.000000
max       24075.000000
Name: tokenized, dtype: float64

In [ ]:
with open('tokenized_df.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df.head()

,Unnamed: 0,text,class,tokenized
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,"[ex, wife, threatening, suicide, ##re, ##cent,..."
1,3,Am I weird I don't get affected by compliments...,non-suicide,"[am, i, weird, i, don, ', t, get, affected, by..."
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,"[finally, 2020, is, almost, over, ., ., ., so,..."
3,8,i need helpjust help me im crying so hard,suicide,"[i, need, help, ##just, help, me, im, crying, ..."
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,"[i, ’, m, so, lost, ##hell, ##o, ,, my, name, ..."


In [ ]:
def tokens_to_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

df['token_ids'] = df['tokenized'].apply(tokens_to_ids)

In [ ]:
all_tokens = [token for sublist in df['token_ids'] for token in sublist]
unique_tokens = set(all_tokens)
vocab_size = len(unique_tokens)
print(vocab_size)

26380


In [ ]:
def padding_and_segmentation(tokens, desired_length=250, pad_token=0):
    length = len(tokens)

    if length < desired_length:
        padded_tokens = tokens + [pad_token] * (desired_length - length)
        return [padded_tokens]

    elif length > desired_length:
        segments = [tokens[i:i + desired_length] for i in range(0, length, desired_length)]

        if len(segments[-1]) < desired_length:
            segments[-1] = segments[-1] + [pad_token] * (desired_length - len(segments[-1]))
        return segments

    else:
        return [tokens]

In [ ]:
# apply function
df['padded_tokens'] = df['token_ids'].apply(padding_and_segmentation)

# make each segment a different row
df = df.explode('padded_tokens', ignore_index=True)

In [ ]:
df.head()

,Unnamed: 0,text,class,tokenized,token_ids,padded_tokens
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,"[ex, wife, threatening, suicide, ##re, ##cent,...","[4654, 2564, 8701, 5920, 2890, 13013, 2135, 10...","[4654, 2564, 8701, 5920, 2890, 13013, 2135, 10..."
1,3,Am I weird I don't get affected by compliments...,non-suicide,"[am, i, weird, i, don, ', t, get, affected, by...","[2572, 1045, 6881, 1045, 2123, 1005, 1056, 213...","[2572, 1045, 6881, 1045, 2123, 1005, 1056, 213..."
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,"[finally, 2020, is, almost, over, ., ., ., so,...","[2633, 12609, 2003, 2471, 2058, 1012, 1012, 10...","[2633, 12609, 2003, 2471, 2058, 1012, 1012, 10..."
3,8,i need helpjust help me im crying so hard,suicide,"[i, need, help, ##just, help, me, im, crying, ...","[1045, 2342, 2393, 29427, 2393, 2033, 10047, 6...","[1045, 2342, 2393, 29427, 2393, 2033, 10047, 6..."
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,"[i, ’, m, so, lost, ##hell, ##o, ,, my, name, ...","[1045, 1521, 1049, 2061, 2439, 18223, 2080, 10...","[1045, 1521, 1049, 2061, 2439, 18223, 2080, 10..."


In [ ]:
len(df)

310601

In [ ]:
df['padded_tokens'].apply(lambda x: len(x)).describe()

count    310601.0
mean        250.0
std           0.0
min         250.0
25%         250.0
50%         250.0
75%         250.0
max         250.0
Name: padded_tokens, dtype: float64

In [ ]:
df.to_csv('/content/drive/My Drive/CSCI2470_final_project/clean.csv', columns=['padded_tokens', 'label'], index=False)

In [ ]:
df_subset = df[['padded_tokens', 'label']]
df_subset.to_pickle('/content/drive/My Drive/CSCI2470_final_project/clean.pkl')

In [ ]:
all_tokens = [token for sublist in df['padded_tokens'] for token in sublist]
unique_tokens = set(all_tokens)
vocab_size = len(unique_tokens)
print(vocab_size)

26381
